### Módulo 2: Ejercicios [Análisis estadísticos]

Un equipo de investigadores está trabajando en la mejora de la calidad del agua contaminada utilizando diferentes concentraciones de reactivos. Han diseñado un experimento en un sistema piloto para evaluar la eficacia de cuatro diferentes tratamientos (T1, T2, T3, y T4). Los parámetros clave analizados son:

Demanda Bioquímica de Oxígeno (DBO): Indica la cantidad de oxígeno requerida para la degradación de materia orgánica. </br>
Concentración de Tensoactivos: Representa los residuos de detergentes y otros compuestos similares. </br>
Sólidos Totales: Mide la cantidad total de partículas suspendidas y disueltas. </br>
pH: Refleja la acidez o alcalinidad del agua tratada. </br>

El equipo realizó cuatro repeticiones aleatorias para cada tratamiento y recopiló datos de cada uno de los parámetros mencionados. El objetivo del experimento es identificar si existen diferencias significativas entre los tratamientos y determinar cuál es el más efectivo.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats
import scipy.stats as stats
import matplotlib.pyplot as plt

#### Importamos los datos del experimento

In [2]:
# Cargar datos desde el archivo CSV
data_path = 'datos_csv/datos_TAR_dca.csv'
df = pd.read_csv(data_path)

#resultados = {}
parametros = ['DBO', 'Tensoactivos', 'Solidos_Totales', 'pH']
tratamientos = df["Tratamiento"].unique()

#print(df.head())
#print(df)

### Estadísticos del experimento

In [8]:
def estadistica_descriptiva(datos):
    df = pd.Series(datos)
    resumen = {
        "Conteo": df.count(),
        "Número de datos únicos": df.nunique(),
        "Moda": df.mode().tolist(),
        "Media": df.mean(),
        "Mediana": df.median(),
        "Varianza": df.var(),
        "Desviación estándar": df.std(),
        #"Coeficiente de variación (%)": (df.std() / df.mean()) * 100 if df.mean() != 0 else np.nan,
        "Mínimo": df.min(),
        "Máximo": df.max(),
        "Rango": df.max() - df.min(),
        "Percentil 25": df.quantile(0.25),
        "Percentil 50 (Mediana)": df.quantile(0.50),
        "Percentil 75": df.quantile(0.75),
    }
    return resumen

# Ejemplo de uso
datos = parametros[0]


# Imprimir resultados
#for treatment in tratamientos:
 #   datos_tratamiento = df[df['Tratamiento'] == treatment][datos]
  #  resultado = estadistica_descriptiva(datos_tratamiento) 
   # for clave, valor in resultado.items():
    #    print(f"{clave}: {valor}")


### Gráficos boxplot

In [ ]:
# Gráficos boxplot
plt.figure(figsize=(12, 8))

# DBO plot
plt.subplot(2, 2, 1)
sns.boxplot(x='Tratamiento', y='DBO', data=df)
plt.title('DBO')

# Tensoactivos plot
plt.subplot(2, 2, 2)
sns.boxplot(x='Tratamiento', y='Tensoactivos', data=df)
plt.title('Tensoactivos')

# Solidos_totales plot
plt.subplot(2, 2, 3)
sns.boxplot(x='Tratamiento', y='Solidos_Totales', data=df)
plt.title('Solidos Totales')

# pH plot
plt.subplot(2, 2, 4)
sns.boxplot(x='Tratamiento', y='pH', data=df)
plt.title('pH')

plt.tight_layout()

##### Analisis de Percentiles

In [3]:
var = parametros[0]

print(f"Quartiles por tratamiento para {var}\n")
for treatment in tratamientos:
    P_q  = df[df['Tratamiento'] == treatment][var]
    perc = float(np.percentile(P_q,[25]))
    print(f"{treatment}:  {perc:.4f}")
    #print(P_q)

Quartiles por tratamiento para DBO

T1:  153.3797
T2:  117.6586
T3:  85.3333
T4:  42.2799


### Análisis estadístico para cada parámetro

#### DBO

In [ ]:
# Verificación de normalidad y homogeneidad para DBO
variable = parametros[0] #DBO

# Verificación de normalidad
print("\nPrueba de normalidad (Shapiro-Wilk) por tratamiento:\n")
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    stat, p_value = stats.shapiro(datos_tratamiento) 
    print(f"{treatment}: Estadístico = {stat:.4f}, p-valor = {p_value:.4f}")

# Verificación de homogeneidad de varianzas (Levene)
stat, p_value = stats.levene(*[df[df['Tratamiento'] == treatment][variable] for treatment in tratamientos])
print(f"\nPrueba de homogeneidad de varianzas (Levene): Estadístico = {stat:.4f}, p-valor = {p_value:.4f}")

# Cálculo de intervalos de confianza para DBO
print("\nIntervalos de confianza (95%) para la media de DBO:\n")
means = []
ci_lower = []
ci_upper = []
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    mean = np.mean(datos_tratamiento)
    sem = stats.sem(datos_tratamiento)  # Error estándar de la media
    ci = stats.t.interval(0.95, len(datos_tratamiento)-1, loc=mean, scale=sem)  # Intervalo de confianza
    #means.append(mean)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])
    print(f"{treatment}: Media = {mean:.4f}, IC 95% = ({ci[0]:.4f}, {ci[1]:.4f})")

In [ ]:
#Gráfico para validar normalidad de resultados
plt.figure()
sns.histplot(df, x=variable, hue='Tratamiento', kde=True, element="bars")
plt.title(f"Histogramas de {variable} por {'Tratamiento'}")
plt.show()
#------------------------------------------------------------------------------------------------------------------------
# Cálculo de residuos
residuos = []
medias_muestrales = []
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    media_tratamiento = datos_tratamiento.mean()
    residuos += list(datos_tratamiento - media_tratamiento)
    medias_muestrales += [media_tratamiento] * len(datos_tratamiento)

#print(medias_muestrales)

##Q-Q plot (Valida normalidad)
plt.figure()
stats.probplot(residuos,plot=plt)
plt.xlabel("Cuantiles Teóricos")
plt.ylabel("Residuos")
plt.grid(True, linestyle='--', alpha=0.6)

#Validar Varianzas (desviaciones muestrales)
plt.figure()
plt.scatter(medias_muestrales, residuos, alpha=0.7, color="orange")
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Medias muestrales")
plt.ylabel("Residuos")
plt.grid(True, linestyle='--', alpha=0.6)
#plt.show()

In [ ]:
# Análisis ANOVA DBO
print("\n--- Análisis para DBO ---")
modelo_dbo = ols('DBO ~ C(Tratamiento)', data=df).fit()
anova_dbo = sm.stats.anova_lm(modelo_dbo)
print(anova_dbo)

tukey_dbo = pairwise_tukeyhsd(df['DBO'], df['Tratamiento'], alpha=0.05)

print("\nTest de Tukey para DBO:")
tukey_dbo.summary()

In [ ]:
#Gráficos para DBO
fig_DBO = tukey_dbo.plot_simultaneous(figsize=(7, 5),
    ylabel="Tratamiento",
    xlabel="Resultados promedios")
plt.title(f"Tukey HSD para {variable}")

# Extraer los resultados del test de Tukey
comparisons = tukey_dbo.summary().data[1:]
comparisons = [f"{comp[0]} vs {comp[1]}" for comp in comparisons]
mean_diffs = [comp[2] for comp in tukey_dbo.summary().data[1:]]
#p_values = [comp[3] for comp in tukey_dbo.summary().data[1:]]

# Crear un gráfico de barras agrupadas por tratamiento
fig, ax = plt.subplots()
bars = ax.bar(comparisons, mean_diffs, color='skyblue')

for bar, mean_diffs in zip(bars, mean_diffs):
    height = bar.get_height() 
    ax.text(bar.get_x() + bar.get_width() / 2, height, 
            f'{mean_diffs:.2f}', ha='center', va='bottom')

ax.set_xlabel('Comparación')
ax.set_ylabel('Diferencia de Medias')
ax.set_title(f"Resultados del Test de Tukey para {variable}")
#plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### Tensoactivos

In [ ]:
# Verificación de normalidad y homogeneidad para Tensoactivos
variable = parametros[1] #Tensoactivos

# Verificación de normalidad
print("\nPrueba de normalidad (Shapiro-Wilk) por tratamiento:\n")
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    stat, p_value = stats.shapiro(datos_tratamiento) 
    print(f"{treatment}: Estadístico = {stat:.4f}, p-valor = {p_value:.4f}")

# Verificación de homogeneidad de varianzas (Levene)
stat, p_value = stats.levene(*[df[df['Tratamiento'] == treatment][variable] for treatment in tratamientos])
print(f"\nPrueba de homogeneidad de varianzas (Levene): Estadístico = {stat:.4f}, p-valor = {p_value:.4f}")

# Cálculo de intervalos de confianza para Tensoactivos
print("\nIntervalos de confianza (95%) para la media de Tensoactivos:\n")
means = []
ci_lower = []
ci_upper = []
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    mean = np.mean(datos_tratamiento)
    sem = stats.sem(datos_tratamiento)  # Error estándar de la media
    ci = stats.t.interval(0.95, len(datos_tratamiento)-1, loc=mean, scale=sem)  # Intervalo de confianza
    means.append(mean)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])
    print(f"{treatment}: Media = {mean:.4f}, IC 95% = ({ci[0]:.4f}, {ci[1]:.4f})")

In [ ]:
#Gráfico para validar normalidad de resultados
plt.figure()
sns.histplot(df, x=variable, hue='Tratamiento', kde=True, element="bars")
plt.title(f"Histogramas de {variable} por {'Tratamiento'}")
plt.show()
#------------------------------------------------------------------------------------------------------------------------
# Cálculo de residuos
residuos = []
medias_muestrales = []
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    media_tratamiento = datos_tratamiento.mean()
    residuos += list(datos_tratamiento - media_tratamiento)
    medias_muestrales += [media_tratamiento] * len(datos_tratamiento)

##Q-Q plot (Valida normalidad)
plt.figure()
stats.probplot(residuos, dist="norm", plot=plt)
plt.xlabel("Cuantiles Teóricos")
plt.ylabel("Residuos")
plt.grid(True, linestyle='--', alpha=0.6)

#Validar Varianzas (desviaciones muestrales)
plt.figure()
plt.scatter(medias_muestrales, residuos, alpha=0.7, color="orange")
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Medias muestrales")
plt.ylabel("Residuos")
plt.grid(True, linestyle='--', alpha=0.6)
#plt.show()

In [ ]:
# ANÁLISIS PARA TENSOACTIVOS
print("\n--- Análisis para Tensoactivos ---")
modelo_tenso = ols('Tensoactivos ~ C(Tratamiento)', data=df).fit()
anova_tenso = sm.stats.anova_lm(modelo_tenso, typ=2)
print(anova_tenso)

tukey_tenso = pairwise_tukeyhsd(df['Tensoactivos'], df['Tratamiento'], alpha=0.05)
print("\nTest de Tukey para Tensoactivos:")
tukey_tenso.summary()

In [ ]:
#Gráficos para tensoactivos

fig_tenso = tukey_tenso.plot_simultaneous(figsize=(7, 5),
    ylabel="Tratamiento",
    xlabel="Resultados promedios")
plt.title("Tukey HSD para Tensoactivos")

# Extraer los resultados del test de Tukey
comparisons = tukey_tenso.summary().data[1:]
comparisons = [f"{comp[0]} vs {comp[1]}" for comp in comparisons]
mean_diffs = [comp[2] for comp in tukey_tenso.summary().data[1:]]

# Crear un gráfico de barras agrupadas por tratamiento
fig, ax = plt.subplots()
bars = ax.bar(comparisons, mean_diffs, color='skyblue')

for bar, mean_diffs in zip(bars, mean_diffs):
    height = bar.get_height() 
    ax.text(bar.get_x() + bar.get_width() / 2, height, 
            f'{mean_diffs:.4f}', ha='center', va='bottom')

ax.set_xlabel('Comparación')
ax.set_ylabel('Diferencia de Medias')
ax.set_title('Resultados del Test de Tukey para Tensoactivos por grupos')
#plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### Sólidos totales

In [ ]:
# Verificación de normalidad y homogeneidad para ST
variable = parametros[2] #ST

# Verificación de normalidad
print("\nPrueba de normalidad (Shapiro-Wilk) por tratamiento:\n")
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    stat, p_value = stats.shapiro(datos_tratamiento) 
    print(f"{treatment}: Estadístico = {stat:.4f}, p-valor = {p_value:.4f}")

# Verificación de homogeneidad de varianzas (Levene)
stat, p_value = stats.levene(*[df[df['Tratamiento'] == treatment][variable] for treatment in tratamientos])
print(f"\nPrueba de homogeneidad de varianzas (Levene): Estadístico = {stat:.4f}, p-valor = {p_value:.4f}")

# Cálculo de intervalos de confianza para ST
print("\nIntervalos de confianza (95%) para la media de ST:\n")
means = []
ci_lower = []
ci_upper = []
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    mean = np.mean(datos_tratamiento)
    sem = stats.sem(datos_tratamiento)  # Error estándar de la media
    ci = stats.t.interval(0.95, len(datos_tratamiento)-1, loc=mean, scale=sem)  # Intervalo de confianza
    means.append(mean)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])
    print(f"{treatment}: Media = {mean:.4f}, IC 95% = ({ci[0]:.4f}, {ci[1]:.4f})")

In [ ]:
#Gráfico para validar normalidad de resultados
plt.figure()
sns.histplot(df, x=variable, hue='Tratamiento', kde=True, element="bars")
plt.title(f"Histogramas de {variable} por {'Tratamiento'}")
plt.show()
#------------------------------------------------------------------------------------------------------------------------
# Cálculo de residuos
residuos = []
medias_muestrales = []
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    media_tratamiento = datos_tratamiento.mean()
    residuos += list(datos_tratamiento - media_tratamiento)
    medias_muestrales += [media_tratamiento] * len(datos_tratamiento)

##Q-Q plot (Valida normalidad)
plt.figure()
stats.probplot(residuos, dist="norm", plot=plt)
plt.xlabel("Cuantiles Teóricos")
plt.ylabel("Residuos")
plt.grid(True, linestyle='--', alpha=0.6)

#Validar Varianzas (desviaciones muestrales)
plt.figure()
plt.scatter(medias_muestrales, residuos, alpha=0.7, color="orange")
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Medias muestrales")
plt.ylabel("Residuos")
plt.grid(True, linestyle='--', alpha=0.6)
#plt.show()

In [ ]:
# ANÁLISIS PARA SÓLIDOS TOTALES
print("\n--- Análisis para Sólidos Totales ---")
modelo_solidos = ols('Solidos_Totales ~ C(Tratamiento)', data=df).fit()
anova_solidos = sm.stats.anova_lm(modelo_solidos, typ=2)
print(anova_solidos)

tukey_solidos = pairwise_tukeyhsd(df['Solidos_Totales'], df['Tratamiento'], alpha=0.05)
print("\nTest de Tukey para Sólidos Totales:")
tukey_solidos.summary()

In [ ]:
#Gráficos para SST
fig_ST = tukey_solidos.plot_simultaneous(figsize=(7, 5),
    ylabel="Tratamiento",
    xlabel="Resultados promedios")
plt.title("Tukey HSD para SST")

# Extraer los resultados del test de Tukey
comparisons = tukey_solidos.summary().data[1:]
comparisons = [f"{comp[0]} vs {comp[1]}" for comp in comparisons]
mean_diffs = [comp[2] for comp in tukey_solidos.summary().data[1:]]

# Crear un gráfico de barras agrupadas por tratamiento
fig, ax = plt.subplots()
bars = ax.bar(comparisons, mean_diffs, color='skyblue')

for bar, mean_diffs in zip(bars, mean_diffs):
    height = bar.get_height() 
    ax.text(bar.get_x() + bar.get_width() / 2, height, 
            f'{mean_diffs:.4f}', ha='center', va='bottom')

ax.set_xlabel('Comparación')
ax.set_ylabel('Diferencia de Medias')
ax.set_title('Resultados del Test de Tukey para SST por grupos')
#plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### pH

In [ ]:
# Verificación de normalidad y homogeneidad para pH
variable = parametros[3] #pH

# Verificación de normalidad
print("\nPrueba de normalidad (Shapiro-Wilk) por tratamiento:\n")
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    stat, p_value = stats.shapiro(datos_tratamiento) 
    print(f"{treatment}: Estadístico = {stat:.4f}, p-valor = {p_value:.4f}")

# Verificación de homogeneidad de varianzas (Levene)
stat, p_value = stats.levene(*[df[df['Tratamiento'] == treatment][variable] for treatment in tratamientos])
print(f"\nPrueba de homogeneidad de varianzas (Levene): Estadístico = {stat:.4f}, p-valor = {p_value:.4f}")

# Cálculo de intervalos de confianza para pH
print("\nIntervalos de confianza (95%) para la media de pH:\n")
means = []
ci_lower = []
ci_upper = []
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    mean = np.mean(datos_tratamiento)
    sem = stats.sem(datos_tratamiento)  # Error estándar de la media
    ci = stats.t.interval(0.95, len(datos_tratamiento)-1, loc=mean, scale=sem)  # Intervalo de confianza
    means.append(mean)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])
    print(f"{treatment}: Media = {mean:.4f}, IC 95% = ({ci[0]:.4f}, {ci[1]:.4f})")

In [ ]:
#Gráfico para validar normalidad de resultados
plt.figure()
sns.histplot(df, x=variable, hue='Tratamiento', kde=True, element="bars")
plt.title(f"Histogramas de {variable} por {'Tratamiento'}")
plt.show()
#------------------------------------------------------------------------------------------------------------------------
# Cálculo de residuos
residuos = []
medias_muestrales = []
for treatment in tratamientos:
    datos_tratamiento = df[df['Tratamiento'] == treatment][variable]
    media_tratamiento = datos_tratamiento.mean()
    residuos += list(datos_tratamiento - media_tratamiento)
    medias_muestrales += [media_tratamiento] * len(datos_tratamiento)

##Q-Q plot (Valida normalidad)
plt.figure()
stats.probplot(residuos, dist="norm", plot=plt)
plt.xlabel("Cuantiles Teóricos")
plt.ylabel("Residuos")
plt.grid(True, linestyle='--', alpha=0.6)

#Validar Varianzas (desviaciones muestrales)
plt.figure()
plt.scatter(medias_muestrales, residuos, alpha=0.7, color="orange")
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel("Medias muestrales")
plt.ylabel("Residuos")
plt.grid(True, linestyle='--', alpha=0.6)
#plt.show()

In [ ]:
# ANÁLISIS PARA PH
print("\n--- Análisis para pH ---")
modelo_pH = ols('pH ~ C(Tratamiento)', data=df).fit()
anova_pH = sm.stats.anova_lm(modelo_pH, typ=2)
print(anova_pH)

tukey_pH = pairwise_tukeyhsd(df['pH'], df['Tratamiento'], alpha=0.05)
print("\nTest de Tukey para pH:")
tukey_pH.summary()

In [ ]:
#Gráficos para pH
tukey_pH.plot_simultaneous(figsize=(7, 5),
    ylabel="Tratamiento",
    xlabel="Resultados promedios")
plt.title("Tukey HSD para pH")

# Extraer los resultados del test de Tukey
comparisons = tukey_pH.summary().data[1:]
comparisons = [f"{comp[0]} vs {comp[1]}" for comp in comparisons]
mean_diffs = [comp[2] for comp in tukey_pH.summary().data[1:]]

# Crear un gráfico de barras agrupadas por tratamiento
fig, ax = plt.subplots()
bars = ax.bar(comparisons, mean_diffs, color='skyblue')

for bar, mean_diffs in zip(bars, mean_diffs):
    height = bar.get_height() 
    ax.text(bar.get_x() + bar.get_width() / 2, height, 
            f'{mean_diffs:.4f}', ha='center', va='bottom')

ax.set_xlabel('Comparación')
ax.set_ylabel('Diferencia de Medias')
ax.set_title('Resultados del Test de Tukey para pH por grupos')
#plt.xticks(rotation=45)
plt.tight_layout()
plt.show()